In [2]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from db_conexion import establecer_conexion

In [7]:
conn, cursor = establecer_conexion()

# Consulta SQL para seleccionar los datos
query = "SELECT * FROM us_accidents"

# Leer los datos en un DataFrame de pandas
df = pd.read_sql_query(query, conn)

Conexion exitosa a la base de datos


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_14760\4066787522.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [8]:
import pandas as pd

# Configurar pandas para mostrar más filas y columnas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


# Mostrar las primeras 20 filas en formato tabular
df.head(20)

,id,source,severity,start_time,end_time,start_lat,start_lng,end_lat,end_lng,distance_mi,description,street,city,county,state,zipcode,country,timezone,airport_code,weather_timestamp,temperature_f,wind_chill_f,humidity_percent,pressure_in,visibility_mi,wind_direction,wind_speed_mph,precipitation_in,weather_condition,amenity,bump,crossing,give_way,junction,no_exit,railway,roundabout,station,stop,traffic_calming,traffic_signal,turning_loop,sunrise_sunset,civil_twilight,nautical_twilight,astronomical_twilight
0,A-2047758,Source2,2.0,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,NaN,NaN,0.000,Accident on LA-19 Baker-Zachary Hwy at Lower Z...,Highway 19,Zachary,East Baton Rouge,LA,70791-4610,US,US/Central,KBTR,2019-06-12 09:53:00,77.0,77.0,62.0,29.92,10.0,NW,5.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day
1,A-4694324,Source1,2.0,2022-12-03 23:37:14,2022-12-04 01:56:53,38.990562,-77.399070,38.990037,-77.398282,0.056,Incident on FOREST RIDGE DR near PEPPERIDGE PL...,Forest Ridge Dr,Sterling,Loudoun,VA,20164-2813,US,US/Eastern,KIAD,2022-12-03 23:52:00,45.0,43.0,48.0,29.91,10.0,W,5.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
2,A-5006183,Source1,2.0,2022-08-20 13:13:00,2022-08-20 15:22:45,34.661189,-120.492822,34.661189,-120.492442,0.022,Accident on W Central Ave from Floradale Ave t...,Floradale Ave,Lompoc,Santa Barbara,CA,93436,US,US/Pacific,KLPC,2022-08-20 12:56:00,68.0,68.0,73.0,29.79,10.0,W,13.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day
3,A-4237356,Source1,2.0,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,43.680574,-92.972223,1.054,Incident on I-90 EB near REST AREA Drive with ...,14th St NW,Austin,Mower,MN,55912,US,US/Central,KAUM,2022-02-21 17:35:00,27.0,15.0,86.0,28.49,10.0,ENE,15.0,0.0,Wintry Mix,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
4,A-6690583,Source1,2.0,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,35.395476,-118.985995,0.046,RP ADV THEY LOCATED SUSP VEH OF 20002 - 726 CR...,River Blvd,Bakersfield,Kern,CA,93305-2649,US,US/Pacific,KBFL,2020-12-04 01:54:00,42.0,42.0,34.0,29.77,10.0,CALM,0.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
5,A-1101469,Source2,2.0,2021-03-29 07:03:58,2021-03-29 08:51:01,42.532082,-70.944267,NaN,NaN,0.000,Accident on Forest St at Lowell St.,Lowell St,Peabody,Essex,MA,01960-4275,US,US/Eastern,KBVY,2021-03-29 06:53:00,42.0,35.0,58.0,29.37,10.0,W,13.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day
6,A-7222249,Source1,2.0,2020-01-14 16:49:23,2020-01-14 20:49:23,42.421280,-123.119450,42.421280,-123.119450,0.000,At OR-99/Exit 45A - Accident.,I-5 N,Gold Hill,Jackson,OR,97525,US,US/Pacific,KMFR,2020-01-14 16:53:00,35.0,35.0,89.0,28.71,10.0,CALM,0.0,0.0,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
7,A-6198239,Source1,2.0,2021-08-13 16:48:00,2021-08-13 19:09:09,30.191010,-85.682508,30.190329,-85.682530,0.047,Incident on CLAREMONT DR near SPRINGHILL CIR D...,Claremont Dr,Panama City,Bay,FL,32405-3534,US,US/Central,KPAM,2021-08-13 16:56:00,90.0,90.0,55.0,30.03,10.0,SW,12.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
8,A-4222549,Source1,2.0,2022-10-12 13:59:30,2022-10-12 15:33:53,32.868947,-96.804018,32.869500,-96.804014,0.038,Incident on PRESTON RD near NORTHWOOD RD Expec...,Preston Rd,Dallas,Dallas,TX,75225,US,US/Central,KDAL,2022-10-12 13:53:00,91.0,91.0,39.0,29.34,10.0,VAR,7.0,0.0,Fair,False,False,False,False,False,False,False,False,True,False,False,False,False,Day,Day,Day,Day
9,A-5924038,Source1,2.0,2021-10-21 07:39:30,2021-10-21 09:24:30,39.717218,-86.124691,39.733478,-86.137021,1.301,Inci

In [11]:
import pandas as pd

# Columnas a eliminar
columns_to_drop = ['id', 'source', 'country', 'description', 'end_lat', 'end_lng', 
                'civil_twilight', 'nautical_twilight', 'astronomical_twilight']

# Eliminación de las columnas
df_cleaned = df.drop(columns=columns_to_drop)

# Imputar valores nulos en columnas numéricas con la media
df_cleaned['temperature_f'].fillna(df_cleaned['temperature_f'].mean(), inplace=True)

# Imputar valores nulos en columnas categóricas con la moda
df_cleaned['weather_condition'].fillna(df_cleaned['weather_condition'].mode()[0], inplace=True)

# Imputar columnas numéricas con la media
num_cols = ['wind_chill_f', 'humidity_percent', 'pressure_in', 'visibility_mi', 'wind_speed_mph', 'precipitation_in']
df_cleaned[num_cols] = df_cleaned[num_cols].apply(lambda col: col.fillna(col.mean()))

# Imputar columna categórica con el valor más frecuente
df_cleaned['wind_direction'] = df_cleaned['wind_direction'].fillna(df_cleaned['wind_direction'].mode()[0])

# Imputar Weather_Timestamp con el valor más cercano (por tiempo)
df_cleaned['weather_timestamp'] = df_cleaned['weather_timestamp'].fillna(method='ffill')

# Eliminar filas que contienen valores nulos
df_cleaned.dropna(inplace=True)

# Contar valores nulos (NaN) en cada columna
nan_counts = df_cleaned.isna().sum()

# Contar valores vacíos ('') en cada columna
empty_counts = (df_cleaned == '').sum()

# Combinar los conteos en un solo DataFrame para visualizar mejor
null_summary = pd.DataFrame({
    'NaN Count': nan_counts,
    'Empty String Count': empty_counts,
    'Total Missing': nan_counts + empty_counts
})

# Mostrar el resumen
print(null_summary)

# Configuración para mostrar más filas y columnas si es necesario
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', None)

# Mostrar el DataFrame en formato tabular
df_cleaned.head(100)


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_14760\1104510428.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['temperature_f'].fillna(df_cleaned['temperature_f'].mean(), inplace=True)
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_14760\1104510428.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

                   NaN Count  Empty String Count  Total Missing
severity                   0                   0              0
start_time                 0                   0              0
end_time                   0                   0              0
start_lat                  0                   0              0
start_lng                  0                   0              0
distance_mi                0                   0              0
street                     0                   0              0
city                       0                   0              0
county                     0                   0              0
state                      0                   0              0
zipcode                    0                   0              0
timezone                   0                   0              0
airport_code               0                   0              0
weather_timestamp          0                   0              0
temperature_f              0            

,severity,start_time,end_time,start_lat,start_lng,distance_mi,street,city,county,state,zipcode,timezone,airport_code,weather_timestamp,temperature_f,wind_chill_f,humidity_percent,pressure_in,visibility_mi,wind_direction,wind_speed_mph,precipitation_in,weather_condition,amenity,bump,crossing,give_way,junction,no_exit,railway,roundabout,station,stop,traffic_calming,traffic_signal,turning_loop,sunrise_sunset
0,2.0,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,0.000,Highway 19,Zachary,East Baton Rouge,LA,70791-4610,US/Central,KBTR,2019-06-12 09:53:00,77.000000,77.000000,62.000000,29.920000,10.000000,NW,5.000000,0.000000,Fair,False,False,False,False,False,False,False,False,False,False,False,True,False,Day
1,2.0,2022-12-03 23:37:14,2022-12-04 01:56:53,38.990562,-77.399070,0.056,Forest Ridge Dr,Sterling,Loudoun,VA,20164-2813,US/Eastern,KIAD,2022-12-03 23:52:00,45.000000,43.000000,48.000000,29.910000,10.000000,W,5.000000,0.000000,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
2,2.0,2022-08-20 13:13:00,2022-08-20 15:22:45,34.661189,-120.492822,0.022,Floradale Ave,Lompoc,Santa Barbara,CA,93436,US/Pacific,KLPC,2022-08-20 12:56:00,68.000000,68.000000,73.000000,29.790000,10.000000,W,13.000000,0.000000,Fair,False,False,False,False,False,False,False,False,False,False,False,True,False,Day
3,2.0,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,1.054,14th St NW,Austin,Mower,MN,55912,US/Central,KAUM,2022-02-21 17:35:00,27.000000,15.000000,86.000000,28.490000,10.000000,ENE,15.000000,0.000000,Wintry Mix,False,False,False,False,False,False,False,False,False,False,False,False,False,Day
4,2.0,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,0.046,River Blvd,Bakersfield,Kern,CA,93305-2649,US/Pacific,KBFL,2020-12-04 01:54:00,42.000000,42.000000,34.000000,29.770000,10.000000,CALM,0.000000,0.000000,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
5,2.0,2021-03-29 07:03:58,2021-03-29 08:51:01,42.532082,-70.944267,0.000,Lowell St,Peabody,Essex,MA,01960-4275,US/Eastern,KBVY,2021-03-29 06:53:00,42.000000,35.000000,58.000000,29.370000,10.000000,W,13.000000,0.000000,Fair,False,False,False,False,False,False,False,False,False,False,False,True,False,Day
6,2.0,2020-01-14 16:49:23,2020-01-14 20:49:23,42.421280,-123.119450,0.000,I-5 N,Gold Hill,Jackson,OR,97525,US/Pacific,KMFR,2020-01-14 16:53:00,35.000000,35.000000,89.000000,28.710000,10.000000,CALM,0.000000,0.000000,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Day
7,2.0,2021-08-13 16:48:00,2021-08-13 19:09:09,30.191010,-85.682508,0.047,Claremont Dr,Panama City,Bay,FL,32405-3534,US/Central,KPAM,2021-08-13 16:56:00,90.000000,90.000000,55.000000,30.030000,10.000000,SW,12.000000,0.000000,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day
8,2.0,2022-10-12 13:59:30,2022-10-12 15:33:53,32.868947,-96.804018,0.038,Preston Rd,Dallas,Dallas,TX,75225,US/Central,KDAL,2022-10-12 13:53:00,91.000000,91.000000,39.000000,29.340000,10.000000,VAR,7.000000,0.000000,Fair,False,False,False,False,False,False,False,False,True,False,False,False,False,Day
9,2.0,2021-10-21 07:39:30,2021-10-21 09:24:30,39.717218,-86.124691,1.301,I-65,Indianapolis,Marion,IN,46237,US/Eastern,KIND,2021-10-21 07:47:00,63.000000,63.000000,78.000000,29.060000,10.000000,SW,10.000000,0.000000,Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
